# Postgres

Stores my raw data and final prepped data hosted at 192.168.68.40

In [ ]:
!pip install psycopg2-binary

In [1]:
import psycopg2
from psycopg2 import sql, pool
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import os
from dotenv import load_dotenv
load_dotenv()
password = os.getenv("DB_PASSWORD")

# Database connection parameters
db_params = {
    'database': 'star_wars_data',
    'user': 'postgres',
    'password': password,
    'host': '192.168.68.40',
    'port': '5432'
}

conn_pool = psycopg2.pool.SimpleConnectionPool(1, 1000, **db_params)


In [3]:
def run_sql(sql_command, data=None):
    try:
        conn = conn_pool.getconn()
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = conn.cursor()
        if data is not None:
            # print("data is not None")
            cursor.execute(sql_command, data)
        else:
            # print("data is None")
            cursor.execute(sql_command)
        result = cursor.fetchall()
        # print(result)
        cursor.close()

        # print("psql completed")
        return result
    except Exception as e:
        print(f"A database error occurred: {e}")
    finally:
        conn_pool.putconn(conn)

checked_urls = set()
def url_exists(url):
    if url in checked_urls:
        return True
    select_query = "SELECT URL FROM all_data WHERE URL = %s"
    result = run_sql(select_query, (url,))
    if len(result) > 0:
        checked_urls.add(url)
        return True
    return False


def preload_checked_urls():
    global checked_urls
    select_query = "SELECT URL FROM all_data"
    result = run_sql(select_query)
    checked_urls = set()
    for row in result:
        checked_urls.add(row[0])
    print(f"Preloaded {len(checked_urls)} URLs")

preload_checked_urls()




Preloaded 223 URLs


In [6]:
import httpx
import asyncio
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import datetime
import time

base_url = "https://starwars.fandom.com/"
start_url = "https://starwars.fandom.com/sitemap-newsitemapxml-index.xml"


ignore_urls = [
    "/wiki/Special:",
    "/wiki/User_talk:",
    "/wiki/Template:",
    "/wiki/Template_talk:",
    "/wiki/Help:",
    "/wiki/User:",
    "/wiki/UserProfile:",
    '/register',
    '/signin',
    '/reset-password']

# get updates from https://starwars.fandom.com/wiki/Special:Log
preload_checked_urls()

total_processed = 0
total_added = 0
total_skipped = 0
errors_urls = []
async def fetch(url):
    global errors_urls
    try:
        async with httpx.AsyncClient(follow_redirects=True, max_redirects=20) as client:
            resp = await client.get(url)
            if resp.status_code == 200:
                content_type = resp.headers.get('Content-Type', '')
                if 'xml' in content_type:
                    return resp, 'xml'
                elif 'html' in content_type:
                    return resp, 'html'
                else:
                    print(f"Unknown content type for {url}: {content_type}")
                    return resp, 'html'

            else:
                raise Exception(f"{url} {resp.status_code}")
    except Exception as e:
        print(f"GET error occurred: {e}")
        errors_urls.append((url,e))
        return None

async def scrape(url, base_url):
    global total_processed
    global total_added
    global total_skipped
    print(f"\nTotal checked: {len(checked_urls)}, processed: {total_processed} in {time.time() - start_time}s (avg {total_processed/(time.time() - start_time)}/s), added: {total_added}, skipped: {total_skipped}")
    print("  Processing:", url)
    response, content_type = await fetch(url)
    # await asyncio.sleep(1)
    if response is None:
        return

    links = []
    if content_type == 'xml':
        soup = BeautifulSoup(response.text, 'lxml-xml')
        links = [loc.text for loc in soup.find_all('loc')]
    else:
        soup = BeautifulSoup(response.text, 'html.parser')
        body = soup.find('body')

        h1_tags = body.find_all(id='firstHeading')
        title = h1_tags[0].text.strip() if h1_tags else "No title"
        content = str(body).replace("\n","").replace("\t","").replace("\r","")

        store_data(title, url, content)

        total_processed += 1

        # links = [urljoin(base_url, link.get('href')) for link in soup.find_all('a')]

    tasks = []
    my_urls = set() # a single page will have the same URL many times
    for link in links:
        url_clean = link
        if '?' in url_clean:
            url_clean = url_clean[:url_clean.index('?')]
        url_clean = url_clean if len(url_clean) <= 250 else url_clean[:250]
        if url_clean not in my_urls:
            my_urls.add(url_clean)
            if not any(ignore_url in url_clean for ignore_url in ignore_urls) and \
              url_clean != url and \
              url_exists(url_clean) == False and \
              base_url in url_clean and \
              base_url != url_clean:
                tasks.append(scrape(url_clean, base_url))
                # await asyncio.sleep(0.5)
                # await scrape(url_clean, base_url)
            else:
                total_skipped += 1

    soup.decompose()
    soup=None

    print(f"Gathering tasks for {url}, count: {len(tasks)}")
    await asyncio.gather(*tasks)
    print(f"Finished processing {url}")
    print(f"Elapsed time: {time.time() - start_time}")



def store_data(title, url, content):
    global total_added
    if not url_exists(url):
        postgres_insert_query = "INSERT INTO all_data (Title, URL, Content, Last_Ingested) VALUES (%s,%s,%s,%s) RETURNING *"
        record_to_insert = (title, url, content, datetime.datetime.now())
        run_sql(postgres_insert_query, record_to_insert)
        print('  Added URL:', url)
        total_added += 1
        checked_urls.add(url)
    else:
        print("  URL already exists in database", url)

start_time = time.time()
await scrape(start_url, base_url)


In [ ]:
print([str(err[0]) for err in errors_urls])




In [ ]:
url_clean = "https://starwars.fandom.com/wiki/Main_Page"
if '?' in url_clean:
    url_clean = url_clean[:url_clean.index('?')]
url_clean = url_clean if len(url_clean) <= 250 else url_clean[:250]
print(url_clean)

In [7]:
select_query = "SELECT URL FROM all_data"
result = run_sql(select_query)
print(len(result))

486
